# Dataframe HowTo with UK-DALE - Daily Analysis

Series and DataFrame objects in pandas are very powerful tools for exploring and analyzing data. Part of their power comes from a multifaceted approach to combining separate datasets, and also performing query for the filter data.

This ipynb is aim to support "daily" UK-DALE analysis, it targets to break down into day, minute(s) or second(s).

**Reference:**
- [Pandas Documentation 0.25.3](https://pandas.pydata.org/pandas-docs/version/0.25.3/getting_started/comparison/comparison_with_sql.html)
- [Pandas Query w3schools](https://www.w3schools.com/python/pandas/ref_df_query.asp)
- [Concatnate Pandas Series](https://www.w3resource.com/pandas/series/series-append.php)
- [Dealing with DateTime Index with Pandas](https://www.sergilehkyi.com/tips-on-working-with-datetime-index-in-pandas/)
        

In [ ]:
import dateutil
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datetime import datetime

import nilmtk as ntk
import util as ut

print("Pandas version: {}".format(pd.__version__))

In [ ]:
plt.rcParams['figure.figsize'] = [15, 10]
RAW_FILENAME = "../Dataset/ukdale.h5"

START_TS ='2013-08-01 00:00:00'
END_TS='2013-08-01 23:59:59'

STR_TITLE_TS = START_TS + " To " + END_TS

HOUSE_NUMBER = 1

# Create object for ukdale
train_ds = ntk.DataSet(RAW_FILENAME)
train_ds.set_window(start=START_TS,end=END_TS)

# Create object for house_data and
# using global variable "HOUSE_NUMBER"
train_house_data = train_ds.buildings[HOUSE_NUMBER].elec

# Mains and Top 5 Appliances

This is to breakdown individual analysis to perform data summary with generating two materialized dataframes.

## Mains - mains_df

### Get mains and load to dataframe - mains_df 

In [ ]:
mains = train_house_data.mains()
raw_df_mains = next(mains.load(sample_period=6))

raw_df_mains.shape

In [ ]:
raw_df_mains.columns

In [ ]:
raw_df_mains.head()

### Flatten Multi-Index columns header and change header name - view_df_main

In [ ]:
view_df_mains = raw_df_mains.copy()

view_df_mains.columns = ["_".join(pair) for pair in view_df_mains.columns]

#view_df_main

# change column name 
view_df_mains = view_df_mains.rename(columns={"power_active":"m_active"})
view_df_mains = view_df_mains.rename(columns={"voltage_":"m_voltage"})
view_df_mains = view_df_mains.rename(columns={"power_apparent":"m_apparent"})

view_df_mains.describe()

### Analysis mains Hourly with Boxplot

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(16, 23))

plot_df_bp = view_df_mains.copy()

plot_df_bp['InHour'] = plot_df_bp.index.strftime('%H')
plot_df_bp = plot_df_bp.set_index('InHour')

plot_df_bp.boxplot(by='InHour', column=['m_active'], ax = axes[0])
plot_df_bp.boxplot(by='InHour', column=['m_voltage'], ax = axes[1])
plot_df_bp.boxplot(by='InHour', column=['m_apparent'], ax = axes[2])

### Resample to Minute & Plot

In [ ]:
# Duplicate dataframe for plotting
plot_df_resample = view_df_mains.resample('1min').mean()

# Handling xtickslabel datetime format
plot_df_resample["TS"] = plot_df_resample.index.strftime('%H:%M:%S')
plot_df_resample = plot_df_resample.set_index('TS')

plot_df_resample.plot(subplots=True, x_compat=True, grid=True)

In [ ]:
plot_df_resample.describe(include = 'all')

### Check whether any null values in columns

In [ ]:
cnt = 0
for column in view_df_main:
    cnt_null = view_df_mains[column].isnull().sum()
    if cnt_null > 0:
        print("{} has {} null row(s).".format(column, cnt_null))
        cnt=+1

if cnt == 0:
    print("Hurray! No null value found in mains_df.")

## Top 5 Appliances

### Get Top 5 appliances and load to dataframe - top_5_appliances_df

In [ ]:
## Select Top-5 Appliances
top_5_house_data = train_house_data.submeters().select_top_k(k=5)

print("\n\nData Type of mg_top is {}.".format(type(top_5_house_data))) 

# Show the selected MeterGroup values
top_5_house_data

In [ ]:
# Load data to dataframe from MeterGroup
raw_df_appliances_top5 = top_5_house_data.dataframe_of_meters()

print("Is there any null value in dataframe = {}.\n".format(raw_df_appliances_top5.isnull().values.any()))

# Change readable column name
raw_df_appliances_top5.columns = train_house_data.get_labels(raw_df_appliances_top5.columns)

# Show dataframe info for MeterGroup
raw_df_appliances_top5.describe()

In [ ]:
raw_df_appliances_top5.shape

In [ ]:
totcolrow = raw_df_appliances_top5.shape
print("There are totla of {} columns and {} rows in the top5 dataframe.".format(totcolrow[1], totcolrow[0]))

for column in raw_df_appliances_top5:
    cnt_null = raw_df_appliances_top5[column].isnull().sum()
    if cnt_null > 0:
        print("{} has {} null row(s).".format(column, cnt_null))

### Check whether any missing data and Fix

In [ ]:
raw_df_appliances_top5[raw_df_appliances_top5.isnull().any(axis=1)]

In [ ]:
# Duplicate raw top5 appliance dataframe to new fix
top5_fix_df = raw_df_appliances_top5.copy()

# Fix by replacing null to zero
top5_fix_df['Kettle'] = top5_fix_df['Kettle'].fillna(0)

### Plot Top 5 Appliances

In [ ]:
# Make a copy of dataframe and materialize for plotting

plot_top5_df = top5_fix_df.copy()

plot_top5_df["TS"] = plot_top5_df.index.strftime('%H:%M:%S')

plot_top5_df = plot_top5_df.set_index('TS')

mytitle = "Analysis of Appliances Power Usage for a day \n" + STR_TITLE_TS
plot_top5_df.plot(title=mytitle, subplots=True)

## Initial Data Analysis Mains and Top 5 Appliances - top5_summary_df

### Summarize top5_appliances_df

In [ ]:
# Initialize a dataframe to store summary
top5_summary_df = pd.DataFrame(columns=['Name', 'MIN', 'AVG', 'MAX', 'SUM'])

# Get values row by row for top 5 appliances
for column in top5_appliances_df:
    tmprow = {'Name' : column, 
              'MIN' : top5_fix_df[column].min(), 
              'AVG' : top5_fix_df[column].mean(),
              'MAX' : top5_fix_df[column].max(),
              'SUM' : top5_fix_df[column].sum()
             }
    
    top5_summary_df = top5_summary_df.append(tmprow, ignore_index=True)

 # Show all data   
top5_summary_df.head((top5_summary_df.shape)[0])

### Summarize  view_df_mains

In [ ]:
for column in view_df_mains:
    tmprow = {'Name' : column, 
              'MIN' : view_df_mains[column].min(), 
              'AVG' : view_df_mains[column].mean(),
              'MAX' : view_df_mains[column].max(),
              'SUM' : view_df_mains[column].sum()
             }
    top5_summary_df = top5_summary_df.append(tmprow, ignore_index=True)

 # Show all data   
top5_summary_df.head((top5_summary_df.shape)[0])

### Findings

Found out that the max value of Washer dryer is 3640, and apparent of mains os 3053.  This is abnormal, as mains max value should be higher than all appliances's max value.

Further investigation is needed to find out the particular records.

### Study record on appliance 2013-08-01 12:21:54+01:00

In [ ]:
# Filter Washer dryer max value
ret_max_rec_app_df = top5_appliances_df.query("`Washer dryer` >= 3640")

ret_max_rec_app_df

In [ ]:
# Appliance record on 2013-08-01 12:21:54+01:00
top5_appliances_df.loc['2013-08-01 12:21:54+01:00']

### Study record on mains 2013-08-01 12:21:54+01:00

In [ ]:
ret_max_rec_m_df = view_df_mains.query("`m_apparent` >= 3053")
ret_max_rec_m_df

In [ ]:
# Appliance record on 2013-08-01 12:21:54+01:00
view_df_mains.loc['2013-08-01 12:31:54+01:00']

# Merging Dataframe 

This is to merge both dataframe of mains and top5_appliances, it named 'merge_df'.  Then, it follows with comparison analysis.

## Validate two index columns 

In [ ]:
index_list_mains = list(tmpdf_mains.index.values)
index_list_top5 = list(tmpdf_top5.index.values)

if(index_list_mains==index_list_top5):
  print("Equal")
else:
  print("Non equal")

## Merge two dataframe - merge_df

In [ ]:
# Duplicate dataframe
tmpdf_mains = view_df_mains.copy()
tmpdf_top5 = top5_fix_df.copy()

# Create a new column and sum of all appliance
tmpdf_top5['TotApp'] =  tmpdf_top5.sum(axis=1)

merge_df = tmpdf_mains.join(tmpdf_top5)

merge_df.shape

In [ ]:
merge_df

## Plot the overview of merge_df with custom columns

In [ ]:
tmpplot_df = merge_df.copy()
tmpplot_df["TS"] = tmpplot_df.index.strftime('%H:%M:%S')
tmpplot_df = tmpplot_df.set_index('TS')

tmpplot_df.plot(title="Power Usage of Active vs Apparent vs Total of Appliances",
              y=["m_active", "m_apparent", "TotApp"])

## Total up the columns of datafame

In [ ]:
sumtot_sr = tmpplot_df.select_dtypes(pd.np.number).sum()

# Return in Series data type
sumtot_sr

In [ ]:
mytitle = "Mains of Active and Apparent vs Top 5 Appliances\n" + STR_TITLE_TS

sumtot_sr[['m_active', 'm_apparent', 'TotApp']].plot(kind='bar',
                                                     title=mytitle)

## Ratio of Top 5 Appliances with Others

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))

sumtot_sr['TotOtherAppWithApperent'] = sumtot_sr['m_apparent'] - sumtot_sr['TotApp']

mytitle = "Ratio of Mains (Apperent) for Top 5 vs Others Appliances \n" + STR_TITLE_TS
mylabel = ['Others Appliances', 'Top 5 Appliances']

sumtot_sr[['TotOtherAppWithApperent', 'TotApp']].plot(kind='pie',
                                                      labels=mylabel, 
                                                      autopct='%1.1f%%',
                                                      explode=[0,0.1],
                                                      shadow=True,
                                                      title=mytitle,
                                                      ax = axes[0])

mytitle = "Ratio of Mains (Active) for Top 5 vs Others Appliances \n" + STR_TITLE_TS
sumtot_sr['TotOtherAppWithActive'] = sumtot_sr['m_active'] - sumtot_sr['TotApp']
sumtot_sr[['TotOtherAppWithActive', 'TotApp']].plot(kind='pie',
                                                    labels=mylabel, 
                                                    autopct='%1.1f%%',
                                                    explode=[0,0.1],
                                                    shadow=True,
                                                    title=mytitle,
                                                    ax = axes[1])

plt.show()

## Get records of Min and Max

In [ ]:
# This method return in pandas.core.series.Series structure
merge_df.loc[merge_df['m_active'].idxmin()]

In [ ]:
# This method return in pandas.core.frame.DataFrame
merge_df.query('m_active == m_active.max()')

In [ ]:
# This method return in pandas.core.frame.DataFrame
merge_df.query('m_voltage == m_voltage.min()')

In [ ]:
# This method return in pandas.core.frame.DataFrame
merge_df.query('m_voltage == m_voltage.max()')

## Validate voltage mains is less than appliances

In [ ]:
ret_df = merge_df.query("m_apparent < TotApp or m_active < TotApp")

ret_df.shape

In [ ]:
ret_df.head()

# Zoom to Details to "merge_df" into seconds

Prior analysis, it detects at "2013-08-01 02:42:00+01:00", the apparent power is less than total of top 5 appliances for the power consumption.  So, it requires detailing the analysis.

## Initialize and duplicate merge_df dataframe

In [ ]:
ts_zoom_start = '2013-08-01 02:41:00'
ts_zoom_end = '2013-08-01 02:43:00'

# Extract only focus time duration; before and after
merge_df_slide = merge_df.copy()
merge_df_slide = merge_df_slide.loc[ts_zoom_start : ts_zoom_end]

## mains - active, apparent VS fridge freezer

In [ ]:
# Extract only interested columns for analysis
columns = ['m_voltage', 'Washer dryer', 'Solar thermal pumping station', 'Kettle', 'Television', 'TotApp']
merge_df_slide.drop(columns, inplace=True, axis=1)

merge_df_slide.shape

### Inital data analysis

In [ ]:
merge_df_slide.describe()

In [ ]:
merge_df_slide.head(merge_df_slide.shape[0])

### Zoom detail plotting

In [ ]:
merge_df_slide["TS"] = merge_df_slide.index.strftime('%H:%M:%S')
merge_df_slide = merge_df_slide.set_index('TS')
mytitle = "Analysis of Mains Active, Apparent & Fridge Freezer \n" + "START = " + ts_zoom_start + "\n" + "END = " + ts_zoom_end

merge_df_slide.plot(title=mytitle)

## mains - voltage VS mains - active, apparent and fridge freezer 

In [ ]:
# Another method to select column(s) from a dataframe
voltage_df = merge_df.copy()
voltage_df = merge_df[["m_voltage"]]
voltage_df = voltage_df.loc[ts_zoom_start : ts_zoom_end]

print("Shape of dataframe - voltage_df : {}".format((voltage_df.shape)))
voltage_df.head(voltage_df.shape[0])

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))

voltage_df["TS"] = voltage_df.index.strftime('%H:%M:%S')
voltage_df = voltage_df.set_index('TS')
mytitle_vol = "Analysis of Mains Voltage \n" + "START = " + ts_zoom_start + "\n" + "END = " + ts_zoom_end

voltage_df.plot(title=mytitle_vol,
                rot=45,
                ax = axes[0])

merge_df_slide.plot(title=mytitle,
                    rot=45,
                    ax = axes[1])

# Workout & Findings

After the completion this notebook, prepare a simple presentation slide to express your summary and new discovery information from here.

- Further enhancement of more detail analysis
- Develop in away that it is reusable